In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import time
from songline import Sendline

# Line Notify access token
line_access_token = 'sIG39uHJuG8YdzWJ57cXvGngIQXhIVpbn43tK6Do2Sx'  # Replace with your Line Notify token

# Initialize Sendline object
line = Sendline(line_access_token)

# List of Thai stock symbols (SET100)
thai_stocks = [
    'ADVANC.BK', 'AEONTS.BK', 'AMATA.BK', 'AOT.BK', 'AP.BK', 'AWC.BK', 'BANPU.BK', 'BBL.BK', 'BCH.BK', 
    'BCP.BK', 'BCPG.BK', 'BDMS.BK', 'BEC.BK', 'BEM.BK', 'BGRIM.BK', 'BH.BK', 'BJC.BK', 'BLA.BK', 'BPP.BK', 
    'BTS.BK', 'CBG.BK', 'CENTEL.BK', 'CHG.BK', 'CK.BK', 'CKP.BK', 'COM7.BK', 'CPALL.BK', 'CPF.BK', 'CPN.BK', 
    'CRC.BK', 'DOHOME.BK', 'EA.BK', 'EGCO.BK', 'EPG.BK', 'ERW.BK', 'GLOBAL.BK', 'GPSC.BK', 
    'GULF.BK', 'HANA.BK', 'HMPRO.BK', 'INTUCH.BK', 'IRPC.BK', 'IVL.BK', 'JMT.BK', 'KBANK.BK', 'KCE.BK', 
    'KKP.BK', 'KTB.BK', 'KTC.BK', 'LH.BK', 'M.BK', 'MAJOR.BK', 'MBK.BK', 'MEGA.BK', 'MINT.BK', 'MTC.BK', 
    'ORI.BK', 'OSP.BK', 'PLANB.BK', 'PRM.BK', 'PSH.BK', 'PTG.BK', 'PTT.BK', 'PTTEP.BK', 'PTTGC.BK', 
    'QH.BK', 'RATCH.BK', 'RS.BK', 'SAWAD.BK', 'SC.BK', 'SCC.BK', 'SCGP.BK', 'SCN.BK', 'SINGER.BK', 'SPALI.BK', 
    'SPRC.BK', 'STA.BK', 'STEC.BK', 'STGT.BK', 'SUPER.BK', 'TASCO.BK', 'TCAP.BK', 'THANI.BK', 'TISCO.BK', 
    'TKN.BK', 'TTB.BK', 'TOA.BK', 'TOP.BK', 'TPIPP.BK', 'TRUE.BK', 'TTB.BK', 'TU.BK', 'VGI.BK', 'WHA.BK'
]

# Fetch stock data
def fetch_stock_data(stock, period='3mo'):
    stock_data = yf.download(stock, period=period)
    return stock_data

# Initialize daily counts and 52-week high tracking
def initialize_52_week_high(date_range):
    return {date: [] for date in date_range}

# Calculate moving averages and 52-week metrics
def calculate_metrics(stock_data):
    stock_data['10MA'] = stock_data['Close'].rolling(window=10).mean()
    stock_data['20MA'] = stock_data['Close'].rolling(window=20).mean()
    stock_data['50MA'] = stock_data['Close'].rolling(window=50).mean()
    stock_data['200MA'] = stock_data['Close'].rolling(window=200).mean()
    
    high_52_week = stock_data['High'].rolling(window=252, min_periods=1).max()
    return stock_data, high_52_week

# Time-check function to run the logic at 16:30 daily
def wait_until_1630():
    while True:
        now = datetime.now()
        target_time = now.replace(hour=16, minute=30, second=0, microsecond=0)
        
        if now > target_time:
            target_time += timedelta(days=1)
        
        sleep_time = (target_time - now).total_seconds()
        print(f"Sleeping for {sleep_time / 60:.2f} minutes until 16:30...")
        time.sleep(sleep_time)

        # When it is 16:30, run the stock analysis
        run_stock_analysis()

# Stock analysis function
def run_stock_analysis():
    date_range = pd.date_range(end=datetime.today(), periods=5).strftime('%Y-%m-%d')
    high_52_week_tracker = initialize_52_week_high(date_range)

    # Analyze each stock for each day
    for stock in thai_stocks:
        try:
            stock_data = fetch_stock_data(stock)
            stock_df, high_52w = calculate_metrics(stock_data)

            for date in date_range:
                if date in stock_df.index:
                    current_price = stock_df.loc[date, 'Close']

                    # Check if the stock hits the 52-week high on this date
                    if current_price >= high_52w.loc[date]:
                        high_52_week_tracker[date].append(stock)

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")

    # Send Line messages for stocks that hit 52-week high each day
    for date, stocks in high_52_week_tracker.items():
        if stocks:
            message = f"Date: {date}, Stocks hitting 3 month high: {', '.join(stocks)}"
            try:
                line.sendtext(message)  # Send the message via Line
                print(f"Sent Line message: {message}")
            except Exception as e:
                print(f"Error sending message: {e}")
        else:
            print(f"Date: {date}, No Stocks hitting 3 month high.")

# Start the loop to check every day at 16:30
wait_until_1630()
